In [61]:
import torch, os
import numpy as np
import matplotlib.pyplot as plt
import torchvision

In [4]:
inject = torchvision.transforms.functional.erase

In [26]:
from models import utils
train_dir = "/net/scratch/hanliu-shared/data/bm/train"
valid_dir = "/net/scratch/hanliu-shared/data/bm/valid"
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=utils.bm_transform())
valid_dataset = torchvision.datasets.ImageFolder(valid_dir, transform=utils.bm_transform())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=10)

In [78]:
nt, nv = len(train_dataset), len(valid_dataset)
ist, isv = [10] * nt, [10]* nv
ht, hv = [5] * nt, [5] * nv
wt, wv = list(range(10, nt+10)), list(range(nt+10, nt+10 + nv))

In [79]:
x_train, x_valid = [], []
for i, b in enumerate(train_dataset):
    x, y = b
    a = inject(x, ist[i], ist[i], ht[i], wt[i], v=0, inplace=False)
    x_train.append(torchvision.transforms.functional.to_pil_image(a))
for i, b in enumerate(valid_dataset):
    x, y = b
    a = inject(x, isv[i], isv[i], hv[i], wv[i], v=0, inplace=False)
    x_valid.append(torchvision.transforms.functional.to_pil_image(a))

In [80]:
train_aug_dir = "/net/scratch/hanliu-shared/data/bm/train_aug"
valid_aug_dir = "/net/scratch/hanliu-shared/data/bm/valid_aug"

In [81]:
i2p = []
for root, _, fnames in sorted(os.walk(train_dir, followlinks=True)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        if path[-4:] == ".pkl":
            continue
        i2p.append(path.replace("train", "train_aug"))
i2p = {i:p for i, p in enumerate(i2p)}
p2i = {v.split("/")[-1]:k for k, v in i2p.items()}

In [82]:
for i, p in i2p.items():
    x_train[i].save(p)

In [83]:
i2p = []
for root, _, fnames in sorted(os.walk(valid_dir, followlinks=True)):
    for fname in sorted(fnames):
        path = os.path.join(root, fname)
        if path[-4:] == ".pkl":
            continue
        i2p.append(path.replace("valid", "valid_aug"))
i2p = {i:p for i, p in enumerate(i2p)}
p2i = {v.split("/")[-1]:k for k, v in i2p.items()}

In [84]:
for i, p in i2p.items():
    x_valid[i].save(p)

In [85]:
sizet = [h*w for h, w in zip(ht, wt)]
sizev = [h*w for h, w in zip(hv, wv)]

In [91]:
dtt = np.array([[np.abs(sizet[i] - sizet[j]) for i in range(nt)] for j in range(nt)])
dvv = np.array([[np.abs(sizev[i] - sizev[j]) for i in range(nv)] for j in range(nv)])
dtv = np.array([[np.abs(sizev[i] - sizet[j]) for i in range(nv)] for j in range(nt)])

In [96]:
pickle.dump(dtt, open("embeds/aug/aug.bm.train.pkl", "wb"))
pickle.dump(dvv, open("embeds/aug/aug.bm.valid.pkl", "wb"))
pickle.dump(dtv, open("embeds/aug/aug.bm.trxvl.pkl", "wb"))